In [2]:
import os
print(os.getcwd())

/workspaces/IC-RNA-2025/Peaks-dataset-article/redes-ensemble-s/mse_sup_newdt copy


In [ ]:
import pandas as pd
import numpy as np
import itertools
import os
from itertools import combinations, islice

 
diretorio_25_model = "/workspaces/IC-RNA-2025/Peaks-dataset-article/redes-ensemble-s/mse_sup_newdt copy/25_model"

diretorio_1000_model = "/workspaces/IC-RNA-2025/Peaks-dataset-article/redes-ensemble-s/mse_sup_newdt copy/1000_model"



file_list_25_model = [os.path.join(diretorio_25_model, f) for f in os.listdir(diretorio_25_model) if f.endswith(".xlsx")]
file_list_1000_model = [os.path.join(diretorio_1000_model, f) for f in os.listdir(diretorio_1000_model) if f.endswith(".xlsx")]

def calcula_mse_sup( conjunto):
    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    Z_pred_total = np.hstack(z_preds)
    Z_media = np.mean(Z_pred_total, axis=1, keepdims=True)
    
    df=pd.read_excel(f"1000_model/1000_model_40_9_1.xlsx")
    Z =df['Z'].values.reshape(-1, 1)

    mse = np.mean((Z - Z_media) ** 2 )

    #print(f"mse: {mse}")
    return mse

def diversity(conjunto):
    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    
    df=pd.read_excel(f"25_model/25_model_40_9_1.xlsx")
    Z =df['Z'].values.reshape(-1, 1)

    
    Z_pred_total = np.hstack(z_preds)
    Z_media = np.mean(Z_pred_total, axis=1, keepdims=True)
    
    error1 = sum((Z - z_pred) ** 2 for z_pred in z_preds)
    error2 = sum((z_pred - Z_media) ** 2 for z_pred in z_preds)
    error = error1 + error2

    return np.mean(error.T)

def mse_var_bias_covar(conjunto):
    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]  
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    Z_pred_total = np.hstack(z_preds)
    Z_media = np.mean(Z_pred_total, axis=1, keepdims=True)
    
    df=pd.read_excel(f"25_model/25_model_40_9_1.xlsx")
    Z =df['Z'].values.reshape(-1, 1)

    var1 = np.mean([(z_pred - Z_media) ** 2 for z_pred in z_preds])
    var = [np.mean((z_pred - Z_media) ** 2) for z_pred in z_preds]
    var = np.mean(np.array(var))

    #print(f"var: {var}")
    #print(f"var1: {var1}")
    #print(var.shape)

    bias=np.mean((Z_media- Z) ** 2 )
    #print(f"bias: {bias}")
    
    M = 10  #quantidade de redes
    cov_sum = 0
    for i in range(M):
        for j in range(M):
            if i != j:
                f_i = z_preds[i]
                f_j = z_preds[j]
                f_i_mean = np.mean(f_i)
                f_j_mean = np.mean(f_j)
                cov_ij = np.mean((f_i - f_i_mean) * (f_j - f_j_mean))
                cov_sum += cov_ij

    covar = cov_sum / (M * (M - 1))
    #print(f"covar: {covar}")

    return ( var, bias, covar)

def filtrar_combinacoes_sincronizadas(file_list_25, file_list_1000, tamanho_conjunto, pasta_destino, limite=1000):
    resultados = []

    indices = list(range(len(file_list_25)))
    combinacoes_indices = islice(combinations(indices, tamanho_conjunto), limite)

    for count, combinacao_indices in enumerate(combinacoes_indices, start=1):
        combinacao_25 = [file_list_25[i] for i in combinacao_indices]
        combinacao_1000 = [file_list_1000[i] for i in combinacao_indices]

        mse = calcula_mse_sup(combinacao_1000)
        erro = diversity(combinacao_25)
        var, bias, covar = mse_var_bias_covar(combinacao_25)
        MSE_var_bias = var + bias**2
        resultado = {
                "Conjunto": combinacao_25,
                "MSE_sup": mse,
                "Erro de Diversidade": erro,
                "var": var,
                "bias": bias,
                "covar": covar,
                "MSE_var_bias": MSE_var_bias
            }
        resultados.append(resultado)
        salvar_resultados([resultado], pasta_destino)

    return resultados


def salvar_resultados(resultados, pasta_destino):
    
    if not os.path.exists(pasta_destino):
        os.makedirs(pasta_destino)
    
    for count, resultado in enumerate(resultados, start=1):
        
        nome_arquivo = f"resultado_{count}.xlsx"
        caminho_arquivo = os.path.join(pasta_destino, nome_arquivo)
    
        
        df = pd.DataFrame([resultado])
        
        
        df.to_excel(caminho_arquivo, index=False)
        print(f"Resultado salvo em: {caminho_arquivo}")


# Definição dos parâmetros
tamanho_conjunto = 10
pasta_destino = "resultados"


resultados_filtrados = filtrar_combinacoes_sincronizadas(
    file_list_25_model,
    file_list_1000_model,
    tamanho_conjunto=10,
    pasta_destino=pasta_destino,
    limite=10  # apenas 1.000 combinações sejam geradas
)

KeyboardInterrupt: 

In [22]:
import pandas as pd
import numpy as np
import itertools
import os
from itertools import combinations, islice

# Diretórios dos modelos
diretorio_25_model = "/workspaces/IC-RNA-2025/Peaks-dataset-article/redes-ensemble-s/mse_sup_newdt copy/25_model"
diretorio_1000_model = "/workspaces/IC-RNA-2025/Peaks-dataset-article/redes-ensemble-s/mse_sup_newdt copy/1000_model"

# Listagem dos arquivos
file_list_25_model = [os.path.join(diretorio_25_model, f) for f in os.listdir(diretorio_25_model) if f.endswith(".xlsx")]
file_list_1000_model = [os.path.join(diretorio_1000_model, f) for f in os.listdir(diretorio_1000_model) if f.endswith(".xlsx")]

# Função para cálculo de MSE usando 1000_model
def calcula_mse_sup(conjunto):
    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    Z_pred_total = np.hstack(z_preds)
    Z_media = np.mean(Z_pred_total, axis=1, keepdims=True)

    df = pd.read_excel("1000_model/1000_model_40_9_1.xlsx")
    Z = df['Z'].values.reshape(-1, 1)

    mse = np.mean((Z - Z_media) ** 2)
    return mse

# Função para cálculo da diversidade usando 25_model
def diversity(conjunto):
    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]

    df = pd.read_excel("25_model/25_model_40_9_1.xlsx")
    Z = df['Z'].values.reshape(-1, 1)

    Z_pred_total = np.hstack(z_preds)
    Z_media = np.mean(Z_pred_total, axis=1, keepdims=True)

    error1 = sum((Z - z_pred) ** 2 for z_pred in z_preds)
    error2 = sum((z_pred - Z_media) ** 2 for z_pred in z_preds)
    error = error1 + error2

    return np.mean(error.T)

# Função para cálculo de variância, viés e covariância usando 25_model
def mse_var_bias_covar(conjunto):
    df_list = [pd.read_excel(arquivo) for arquivo in conjunto]
    z_preds = [df['Z_pred'].values.reshape(-1, 1) for df in df_list]
    Z_pred_total = np.hstack(z_preds)
    Z_media = np.mean(Z_pred_total, axis=1, keepdims=True)

    df = pd.read_excel("25_model/25_model_40_9_1.xlsx")
    Z = df['Z'].values.reshape(-1, 1)

    var = [np.mean((z_pred - Z_media) ** 2) for z_pred in z_preds]
    var = np.mean(np.array(var))

    bias = np.mean((Z_media - Z) ** 2)

    M = 10  # número de redes
    cov_sum = 0
    for i in range(M):
        for j in range(M):
            if i != j:
                f_i = z_preds[i]
                f_j = z_preds[j]
                f_i_mean = np.mean(f_i)
                f_j_mean = np.mean(f_j)
                cov_ij = np.mean((f_i - f_i_mean) * (f_j - f_j_mean))
                cov_sum += cov_ij

    covar = cov_sum / (M * (M - 1))
    return (var, bias, covar)

# Função principal de geração e avaliação das combinações sincronizadas
def filtrar_combinacoes_sincronizadas(file_list_25, file_list_1000, tamanho_conjunto, pasta_destino, limite=1000):
    resultados = []

    indices = list(range(len(file_list_25)))
    combinacoes_indices = islice(combinations(indices, tamanho_conjunto), limite)

    for count, combinacao_indices in enumerate(combinacoes_indices, start=1):
        combinacao_25 = [file_list_25[i] for i in combinacao_indices]
        combinacao_1000 = [file_list_1000[i] for i in combinacao_indices]

        mse = calcula_mse_sup(combinacao_1000)
        erro = diversity(combinacao_25)
        var, bias, covar = mse_var_bias_covar(combinacao_25)
        MSE_var_bias = var + bias**2

        resultado = {
            "Conjunto": ", ".join(os.path.basename(p) for p in combinacao_25),
            "MSE_sup": mse,
            "Erro de Diversidade": erro,
            "var": var,
            "bias": bias,
            "covar": covar,
            "MSE_var_bias": MSE_var_bias
        }
        resultados.append(resultado)

    salvar_resultados(resultados, pasta_destino)
    return resultados

# Função para salvar todos os resultados de uma vez
def salvar_resultados(resultados, pasta_destino):
    if not os.path.exists(pasta_destino):
        os.makedirs(pasta_destino)

    df = pd.DataFrame(resultados)
    caminho_arquivo = os.path.join(pasta_destino, "todos_resultados.xlsx")
    df.to_excel(caminho_arquivo, index=False)
    print(f"Todos os resultados salvos em: {caminho_arquivo}")

# Parâmetros
tamanho_conjunto = 10
pasta_destino = "resultados"

# Execução
resultados_filtrados = filtrar_combinacoes_sincronizadas(
    file_list_25_model,
    file_list_1000_model,
    tamanho_conjunto=10,
    pasta_destino=pasta_destino,
    limite=10
)


Todos os resultados salvos em: resultados/todos_resultados.xlsx


In [ ]:
import os
import pandas as pd


diretorio1 = "/workspaces/IC-RNA-2025/Peaks-dataset-article/redes-ensemble-s/mse_sup_newdt copy/resultados_filtrados_erro"
diretorio2 = "/workspaces/IC-RNA-2025/Peaks-dataset-article/redes-ensemble-s/mse_sup_newdt copy/resultados_filtrados_mse"
diretorio3 = "/workspaces/IC-RNA-2025/Peaks-dataset-article/redes-ensemble-s/mse_sup_newdt copy/resultados_filtrados_bias"
diretorio4 = "/workspaces/IC-RNA-2025/Peaks-dataset-article/redes-ensemble-s/mse_sup_newdt copy/resultados_filtrados_covar"
diretorio5 = "/workspaces/IC-RNA-2025/Peaks-dataset-article/redes-ensemble-s/mse_sup_newdt copy/resultados_filtrados_var"

file_list1 = [os.path.join(diretorio1, f) for f in os.listdir(diretorio1) if f.endswith(".xlsx")]
file_list2 = [os.path.join(diretorio2, f) for f in os.listdir(diretorio2) if f.endswith(".xlsx")]
file_list3 = [os.path.join(diretorio3, f) for f in os.listdir(diretorio3) if f.endswith(".xlsx")]
file_list4 = [os.path.join(diretorio4, f) for f in os.listdir(diretorio4) if f.endswith(".xlsx")]
file_list5 = [os.path.join(diretorio5, f) for f in os.listdir(diretorio5) if f.endswith(".xlsx")]


def combinar_excel(diretorio, file_list, id_saida):
    df_consolidado = pd.DataFrame()

    for file in file_list:
        try:
            df = pd.read_excel(file)
            df_consolidado = pd.concat([df_consolidado, df], ignore_index=True)
        except Exception as e:
            print(f"Erro ao ler '{file}': {e}")

    caminho_saida = os.path.join(diretorio, f"resultado_consolidado_{id_saida}.xlsx")
    df_consolidado.to_excel(caminho_saida, index=False)
    print(f"[{id_saida}] Arquivo consolidado salvo em: {caminho_saida}")


combinar_excel(diretorio1, file_list1, "erro")
combinar_excel(diretorio2, file_list2, "mse")
combinar_excel(diretorio3, file_list3, "bias")
combinar_excel(diretorio4, file_list4, "covar")
combinar_excel(diretorio5, file_list5, "var")


Erro ao ler '/workspaces/IC-RNA-2025/Peaks-dataset-article/redes-ensemble-s/mse_sup_newdt copy/resultados_filtrados_erro/resultado_845.xlsx': File is not a zip file
[erro] Arquivo consolidado salvo em: /workspaces/IC-RNA-2025/Peaks-dataset-article/redes-ensemble-s/mse_sup_newdt copy/resultados_filtrados_erro/resultado_consolidado_erro.xlsx
[mse] Arquivo consolidado salvo em: /workspaces/IC-RNA-2025/Peaks-dataset-article/redes-ensemble-s/mse_sup_newdt copy/resultados_filtrados_mse/resultado_consolidado_mse.xlsx
[bias] Arquivo consolidado salvo em: /workspaces/IC-RNA-2025/Peaks-dataset-article/redes-ensemble-s/mse_sup_newdt copy/resultados_filtrados_bias/resultado_consolidado_bias.xlsx
[covar] Arquivo consolidado salvo em: /workspaces/IC-RNA-2025/Peaks-dataset-article/redes-ensemble-s/mse_sup_newdt copy/resultados_filtrados_covar/resultado_consolidado_covar.xlsx
[var] Arquivo consolidado salvo em: /workspaces/IC-RNA-2025/Peaks-dataset-article/redes-ensemble-s/mse_sup_newdt copy/resultado